In [28]:
%load_ext autoreload
%autoreload 2

import os

if 'cachai' not in os.listdir('.'):
    os.chdir('../')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
from enum import Enum, auto


class ObservationType(Enum):
    HIT = auto()
    MISS = auto()
    STALE = auto()
    VALID_TTL = auto()

    def __str__(self):
        return self.name

In [30]:
import numpy as np
import pandas as pd
from sklearn.metrics import root_mean_squared_error, mean_absolute_error, f1_score


class TTLSimulator:

    def __init__(self, iterations=1_000):
        self._iterations = iterations
        self._target_params = [
            (50, 5),
            (200, 10),
            (400, 30),
        ]
        # means = np.linspace(10, 500, 10).astype(int)
        # std = np.arange(1, len(means) + 1)**2
        # self._target_params = np.array([means, std]).T

    def update_target_params(self, progress):
        target_params = []
        for param in self._target_params:
            mean = float(
                round(param[0]*np.sin(progress*2*np.pi/2)/(param[0]/2) + param[0], 2)
            )
            std = param[1]
            target_params.append((mean, std))
        self._target_params = target_params

    def feedback(self, y_true, y_pred):
        observation_time = int(min(y_true, y_pred)[0])
        hits = max(0, observation_time-1)
        observation_type = None
        if y_pred[0] < y_true[0]:
            observation_type = ObservationType.MISS
        elif y_pred[0] > y_true[0]:
            observation_type = ObservationType.STALE
        else:
            observation_type = ObservationType.VALID_TTL
        return observation_time, observation_type, hits

    def generate(self):
        # get target
        target_param_index = np.random.randint(0, len(self._target_params))
        target_params = self._target_params[target_param_index]
        y = np.random.normal(target_params[0], target_params[1], 1)

        # generate features from target
        num_features = 1
        correlation = 0.8
        cov_matrix = np.eye(num_features) * (1 - correlation) + np.ones((num_features, num_features)) * correlation
        features = np.random.multivariate_normal(np.ones(num_features) * y, cov_matrix)
        X = features.reshape(1, -1)
        # X = np.full((1, num_features), fill_value=target)
        return X, y

In [31]:
# OTHERS:
# Byte Hit Rate (BHR): The ratio of the number of bytes served from the cache to the total number of bytes requested.
# - BHR = (Total Bytes - Bytes Served) / Total Bytes
# Request Rate (RR): The rate at which requests are being made to the cache.
# - RR = TR / Time
# Latency Metrics:
# - Average Latency: The average time taken to serve a request from the cache.
# - 95th Percentile Latency: The time within which 95% of the requests are served.
# Cache Utilization (CU): The percentage of cache storage being used.
# - CU = (Used Cache Size / Total Cache Size) * 100
# Cache Hit Latency: The time taken to serve a request that results in a cache hit. Use: To ensure that cache hits are served efficiently.
# Miss Rate Decay: Definition: The rate at which the miss rate decreases over time as the cache gets populated. Use: To understand how quickly the cache becomes effective.

from sklearn.calibration import check_consistent_length


def mean_bias_error(y_true, y_pred, fn, *, weights=None):
    check_consistent_length(y_true, y_pred, weights)
    differences = y_pred - y_true
    return np.average(fn(differences), weights=weights)


def mean_squared_bias_error(y_true, y_pred, *, weights=None):
    return mean_bias_error(y_true, y_pred, np.square, weights=weights)


def mean_absolute_bias_error(y_true, y_pred, *, weights=None):
    return mean_bias_error(y_true, y_pred, np.abs, weights=weights)


def evaluate(df):
    """
    - True Positives (TP) as valid cache hits (True Hits).
    - False Positives (FP) as stale cache hits (False Hits).
    - False Negatives (FN) as cache misses (Misses).
    - True Negatives (TN) as valid cache misses (True Misses).
    - Total requests (TR) as the total number of requests.

    - Hit Rate = (TP + FP) / TR — How many of the requests were served from cache
    - Miss Rate = (FN + TN) / TR — How many of the requests were not served from cache
    - Precision = TP / (TP + FP) — How accurate were the cache hits
    - Accuracy = (TP + TN) / TR — How many of the requests were served correctly
    - Recall = TP / (TP + FN) — How many of the cache hits were found
    - FP Rate = FP / (FP + TP) — How many of the cache hits were stale
    - F1 Score = 2 * (Precision * Recall) / (Precision + Recall)
    """

    metrics = []
    y_true = df['y_true']
    y_pred = df['y_pred']
    hits = df['hits']
    hits_total = hits.sum()
    hits_mean = hits.mean()
    total_requests = len(df)
    true_positive = len(df[df['observation_type'] == ObservationType.HIT.name])
    true_negative = len(df[df['observation_type'] == ObservationType.VALID_TTL.name])
    false_positive = len(df[df['observation_type'] == ObservationType.STALE.name])
    false_negative = len(df[df['observation_type'] == ObservationType.MISS.name])

    rmse = root_mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    mabe = mean_absolute_bias_error(y_true, y_pred)
    msbe = mean_squared_bias_error(y_true, y_pred)
    cache_hits = true_positive + false_positive
    not_cache_hits = true_negative + false_negative
    hit_rate = cache_hits / total_requests
    miss_rate = not_cache_hits / total_requests
    accuracy = (true_positive + true_negative) / total_requests
    precision = (true_positive / cache_hits) if cache_hits else 0
    recall = (true_positive / (true_positive + false_negative)) if (true_positive + false_negative) else 0
    false_positive_rate = (false_positive / cache_hits) if cache_hits else 0
    f1_score = (2 * (precision * recall) / (precision + recall)) if (precision + recall) else 0

    metrics.append([
        rmse, mae, mabe, msbe,
        hit_rate, miss_rate, precision, accuracy, recall,
        false_positive_rate, f1_score, hits_total, hits_mean, len(df)
    ])
    return pd.DataFrame(metrics, columns=[
        'rmse', 'mae', 'mabe', 'msbe',
        'hit_rate', 'miss_rate', 'precision', 'accuracy', 'recall',
        'false_positive_rate', 'f1_score', 'hits_total', 'hits_mean', 'len'
    ])


def evaluate_groupby(df, groupby=['experiment_name', 'model_name']):
    return df.groupby(groupby).apply(evaluate, include_groups=False)

In [32]:
class Experiment():

    DF_COLUMNS = [
        'experiment_name', 'model_name', 'iteration', 'observation_type', 'observation_time',
        'y_true', 'y_pred', 'hits', 'mae'
    ]

    def __init__(self, simulator, iterations=1_000):
        self._simulator = simulator
        self._iterations = iterations

    def run(self, experiments):
        df = []
        for experiment in experiments:
            experiment_name = experiment['name']
            model = experiment['model']
            for i in range(self._iterations):
                X, y_true = self._simulator.generate()
                y_pred = model.predict(X)
                observation_time, observation_type, hits = self._simulator.feedback(y_true, y_pred)
                mae = mean_absolute_error(y_true, y_pred)
                model.observe(observation_time, observation_type, hits, y_pred)
                df.append([
                    experiment_name, model.NAME, i, observation_type, observation_time,
                    y_pred[0], y_true[0], hits, mae
                ])
        return pd.DataFrame(df, columns=Experiment.DF_COLUMNS)

In [33]:
from abc import ABC, abstractmethod


class BaseModel(ABC):

    @property
    @abstractmethod
    def NAME(self) -> str:
        pass

    @abstractmethod
    def predict(self, X: np.array) -> np.array:
        pass

    @abstractmethod
    def observe(
        self,
        observation_time: int,
        observation_type: ObservationType,
        hits: int,
        prev_prediction: float
    ) -> None:
        pass

In [34]:
class Model(BaseModel):
    NAME = 'OPTIMAL'

    def __init__(self):
        pass

    def predict(self, X):
        output = X.mean()
        return np.array([output])

    def observe(self, observation_time, observation_type, hits, prev_prediction):
        a = 2+2

In [35]:
simulator = TTLSimulator()
experiment = Experiment(simulator=simulator)
df = experiment.run([
    {
        'name': 'TEST 1',
        'model': Model(),
    },
    {
        'name': 'TEST 2',
        'model': Model(),
    }
])
df.head(10)

,experiment_name,model_name,iteration,observation_type,observation_time,y_true,y_pred,hits,mae
0,TEST 1,OPTIMAL,0,STALE,391,392.188258,391.519394,390,0.668864
1,TEST 1,OPTIMAL,1,MISS,219,219.970528,220.457727,218,0.487199
2,TEST 1,OPTIMAL,2,STALE,49,50.814361,49.988714,48,0.825647
3,TEST 1,OPTIMAL,3,MISS,203,203.234974,204.228549,202,0.993575
4,TEST 1,OPTIMAL,4,MISS,210,210.874697,211.751594,209,0.876898
5,TEST 1,OPTIMAL,5,MISS,393,393.789725,393.798041,392,0.008315
6,TEST 1,OPTIMAL,6,STALE,386,388.193761,386.357318,385,1.836443
7,TEST 1,OPTIMAL,7,STALE,45,46.295637,45.408883,44,0.886754
8,TEST 1,OPTIMAL,8,MISS,191,191.436216,191.987764,190,0.551548
9,TEST 1,OPTIMAL,9,MISS,458,458.007442,458.589182,457,0.581740


In [36]:
evaluate(df)

ValueError: 14 columns passed, passed data had 15 columns

In [43]:
evaluate_groupby(df)

,,,rmse,mae,mabe,msbe,hit_rate,miss_rate,precision,accuracy,recall,false_positive_rate,f1_score,hits_total,hits_mean,len
experiment_name,model_name,,,,,,,,,,,,,,,
TEST 1,OPTIMAL,0,1.020490,0.828314,0.035535,0.044866,0.0,0.0,0,0.0,0,0,0,210465,210.465,1000
TEST 2,OPTIMAL,0,1.041727,0.835591,-0.043121,-0.085409,0.0,0.0,0,0.0,0,0,0,220912,220.912,1000
